In [138]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np

In [3]:
target = pd.read_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1_Targets.csv",
                    sep = ',',
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

,TableID,ColumnID,RowID
0,50245608_0_871275842592178099,0,154
1,39107734_2_2329160387535788734,1,32
2,22864497_0_8632623712684511496,0,227


In [140]:
answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

counter = 0
try:
for index, row in target.iterrows():
    try:
    #     counter = counter + 1
    #     if counter == 10:
    #         break


        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']

        answers_dict['TableID'].append(table_id)
        answers_dict['ColumnID'].append(column_id)
        answers_dict['RowID'].append(row_id)

        df = pd.read_csv(f"D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\{table_id}.csv",
                    sep=',')
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]

        if cell_content is np.nan:
            print(table_id, column_id, row_id)
            print(cell_content)
            continue
        query = """SELECT ?resource WHERE { ?resource rdfs:label """ + '"' + str(cell_content) + '"' + '@' + detect(cell_content) + '}'
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        # sprawdzenie czy w ogóle znaleziono jakikolwiek link do dbpedii
        if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
            answers_dict['URI'].append("BRAK")
            continue

        # jeśli tak to szukamy konkretnie tego linku
        results_list = results['results']['bindings']
        for result_dict in results_list:
    #         print(result_dict)
            uri = result_dict['resource']['value']
            match = re.match("http://dbpedia.org/resource/*", uri)
            if match:
                answers_dict['URI'].append(uri)
                break



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)

54719588_0_8417197176086756912 3 248
nan


LangDetectException: No features in text.

In [ ]:
result_df.to_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\my_submission.csv",
                 index=False,
                 sep=',',
                header=False)

In [66]:
df = pd.read_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\8468806_0_4382447409703007384.csv",
                sep=',')
df[:3]

,Rang,Name,Staat(en),Fläche-(km2),Länge-(km),Max.-Tiefe-(m),Volumen-(km?)
0,1.0,Kaspisches-Meer,Aserbaidschan Russland Kasachstan Turkmenistan...,371.000,1199,1025,78.200
1,2.0,Oberer-See,USA-Kanada,82.414,616,405,12.100
2,3.0,Victoriasee,Kenia Tansania Uganda,69.485,322,84,2750


In [67]:
detect(df.iloc[0:1, 1:2].values[0][0])

'de'

In [68]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

{'head': {'link': [], 'vars': ['label']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'label': {'type': 'literal', 'xml:lang': 'en', 'value': 'Norway'}}, {'label': {'type': 'literal', 'xml:lang': 'ar', 'value': 'النرويج'}}, {'label': {'type': 'literal', 'xml:lang': 'de', 'value': 'Norwegen'}}, {'label': {'type': 'literal', 'xml:lang': 'es', 'value': 'Noruega'}}, {'label': {'type': 'literal', 'xml:lang': 'fr', 'value': 'Norvège'}}, {'label': {'type': 'literal', 'xml:lang': 'it', 'value': 'Norvegia'}}, {'label': {'type': 'literal', 'xml:lang': 'ja', 'value': 'ノルウェー'}}, {'label': {'type': 'literal', 'xml:lang': 'nl', 'value': 'Noorwegen'}}, {'label': {'type': 'literal', 'xml:lang': 'pl', 'value': 'Norwegia'}}, {'label': {'type': 'literal', 'xml:lang': 'pt', 'value': 'Noruega'}}, {'label': {'type': 'literal', 'xml:lang': 'ru', 'value': 'Норвегия'}}, {'label': {'type': 'literal', 'xml:lang': 'zh', 'value': '挪威'}}]}}
Norway
النرويج
Norwegen
Noruega
Norvège
Norvegia
ノルウェー
No

In [69]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

{'head': {'link': [], 'vars': ['a', 'c']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'a': {'type': 'uri', 'value': 'http://dbpedia.org/resource/IKEA'}}]}}


In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

{'head': {'link': []}, 'boolean': True}


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



{'head': {'link': []}, 'boolean': False}


In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [123]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

[{'resource': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5484'}}, {'resource': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Caspian_Sea'}}]
jest


In [101]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())

http://dbpedia.org/resource/
